In [1]:
import pandas as pd
import numpy as np
import requests
import json
import json
import math
import csv
# import seaborn as sns
import time

In [2]:
#Quick way to execute GET requests
api_key = "RGAPI-a0e0d5f3-801c-43d3-a182-02ae0bc815dc"
def get_req(url, params):
    headers = {
                "X-Riot-Token": api_key,
                }
    response = requests.request("GET", url, headers=headers, params=params)
    return(response.text)

In [3]:
#get a recent match list leveraging an offset starting index 
#e.g. requesting 10 matches with offset 0, gets the 10 most recent. 
#e.g requesting 10 matches with offset/start index 5 will skip the first 5 most recent and then grab the next 10
def get_matches(count, puuid, start): 
    qtype = "ranked"
    queue = 420
    url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" + str(puuid) + f'/ids?queue={queue}'
    url+= f'&type={qtype}&start={start}&count={count}'
    res = get_req(url=url, params=None)
    match_list = res
    return json.loads(match_list)
    


In [4]:
#get the gigantic match DTO
def get_match_analytics(match_id):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}'
    res = json.loads(get_req(url, params = None))
    return res

In [5]:
#lets carve out a really big data frame :)
df = pd.DataFrame()
#solo stats
df['match-id'] = ''
df['game-date'] = ''
df['game-type'] = ''
#challenges
df['assist-me-pings'] = ''
df['enemy-missing-pings'] = ''
df['bounty-level'] = ''
df['ability-uses'] = ''
df['bounty-gold'] = ''
df['buffs-stolen'] = ''
df['control-wards-placed'] = ''
df['damage-per-minute'] = ''
df['damage-taken-on-team-%'] = ''
df['deaths-by-enemy-champ'] = ''
df['early-laning-gold-exp-advantage'] = ''
df['epic-monster-steals'] = ''
df['first-turret-killed'] = ''
df['gold-per-minute'] = ''
df['kda'] = ''
df['kpa'] = ''
df['max-cs-advantage-on-lane-opponent'] = ''
df['max-kill-deficit'] = ''
df['outnumbered-kills'] = ''
df['played-champ-select-position'] = ''
df['takedowns'] = ''

df['turret-plates-taken'] = ''
df['turret-damage-dealt'] = ''
df['vision-score-per-minute'] = ''
df['vision-score'] = ''
df['champ-name'] = ''
df['champ-level'] = ''
df['deaths'] = ''
df['first-blood'] = ''
df['first-blood-assist'] = ''
df['gold-earned'] = ''
df['gold-spent'] = ''
df['individual-position'] = ''
df['inhibitors-lost'] = ''
df['kills'] = ''
df['longest-time-alive'] = ''
df['riot-id-game-name'] = ''
df['team-position'] = ''
df['total-damage-dealt'] = ''
df['total-time-spent-dead'] = ''
df['team-id'] = ''
df['win'] = ''
df['game-duration'] = ''
df['game-mode'] = 'Ranked'

In [6]:
#this is the meat and potatoes
#consumes the match DTO and creates a row per player in a given match 
#grabbing the columns per player shown below
def fill_df_with_match_info(df, match, match_id):
    try:
        participants = match['info']['participants']
        for player in participants:
            row = {'game-duration': match['info']['gameDuration'],
                'game-date': str(match['info']['gameCreation']),
                'game-type': str(match['info']['gameType']),
                'match-id': match_id,
                'total-time-spent-dead': player['totalTimeSpentDead'],
                'win': player['win'],
                'team-id': player['teamId'],
                'total-damage-dealt': player['totalDamageDealtToChampions'],
                'riot-id-game-name': player['riotIdGameName'],
                'game-mode': match['info']['gameMode'],
                'team-position': player['teamPosition'],
                'longest-time-alive': player['longestTimeSpentLiving'],
                'kills': player['kills'],
                'inhibitors-lost': player['inhibitorsLost'],
                'individual-position': player['individualPosition'],
                'gold-spent': player['goldSpent'],
                'gold-earned': player['goldEarned'],
                'first-blood-assist': player['firstBloodAssist'],
                'first-blood': player['firstBloodKill'],
                'deaths': player['deaths'],
                'champ-level': player['champLevel'],
                'champ-name': player['championName'],
                'vision-score': player['visionScore'],
                'vision-score-per-minute': player['challenges']['visionScorePerMinute'],
                'turret-plates-taken': player['challenges']['turretPlatesTaken'],
                'turret-damage-dealt': player['damageDealtToTurrets'],
                'takedowns': player['challenges']['takedowns'],
#                 'played-champ-select-position': player['challenges']['playedChampSelectPosition'],
                'outnumbered-kills': player['challenges']['outnumberedKills'],
                'max-kill-deficit': player['challenges']['maxKillDeficit'],
                'max-cs-advantage-on-lane-opponent': player['challenges']['maxCsAdvantageOnLaneOpponent'],
                'kpa': player['challenges']['killParticipation'],
                'kda': player['challenges']['kda'],
                'gold-per-minute': player['challenges']['goldPerMinute'],
                'first-turret-killed': player['challenges']['firstTurretKilled'],
                'epic-monster-steals': player['challenges']['epicMonsterSteals'],
                'early-laning-gold-exp-advantage': player['challenges']['earlyLaningPhaseGoldExpAdvantage'],
                'deaths-by-enemy-champ': player['challenges']['deathsByEnemyChamps'],
                'damage-taken-on-team-%': player['challenges']['damageTakenOnTeamPercentage'],
                'damage-per-minute': player['challenges']['damagePerMinute'],
                'control-wards-placed': player['challenges']['controlWardsPlaced'],
                'buffs-stolen': player['challenges']['buffsStolen'],
                'bounty-gold': player['challenges']['bountyGold'],
                'ability-uses': player['challenges']['abilityUses'],
                'bounty-level': player['bountyLevel'],
                'assist-me-pings': player['assistMePings'],
                'enemy-missing-pings': str(player['enemyMissingPings'])
            }
            df_dictionary = pd.DataFrame([row])
            df = pd.concat([df, df_dictionary], ignore_index=True)
    except Exception as e:
        print(e)
    return df


In [7]:
#just a shortcut to get a lot of matches.
#total is the total number of matches to grab in increments of 100 as per the api spec 
def get_matches_bulk():
    total = 1000
    query_offset = 0
    match_list = []
    while(total > query_offset):
        res = get_matches(100, puuid, query_offset)
        print(len(match_list), end = ' | ')
        query_offset += 100
        match_list.extend(res)
    return match_list

In [11]:
#this gets account puuid: 
player = 'Liltimy'
account_v1 = f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{player}'
res = json.loads(get_req(account_v1, params = None))
puuid = res['puuid']
print(puuid)

YuY7Q7ufbliP-IULanP0ye0x2fbZG7oLahBBKMyoEiHZid4M9F_cjVTspD9bmu7xaJg3552bDT3JQw


In [12]:
match_set = get_matches_bulk()
print("\ndone finding matches")
#clean up the match list
match_list = []
for match in match_set:
    if str(match) not in match_list:
        match_list.append(str(match))

print("found: ", len(match_list))

0 | 100 | 200 | 300 | 400 | 422 | 422 | 422 | 422 | 422 | 
done finding matches
found:  422


In [13]:
#match analysis
#get players matches
count = 1
for match in match_list:
    data = get_match_analytics(match)
    df = fill_df_with_match_info(df, data, match)
    print(count)
    count+=1

1
2
3
4
5
'riotIdGameName'
6
'riotIdGameName'
7
'riotIdGameName'
8
'riotIdGameName'
9
'riotIdGameName'
10
'riotIdGameName'
11
'riotIdGameName'
12
'riotIdGameName'
13
'riotIdGameName'
14
'riotIdGameName'
15
'riotIdGameName'
16
'riotIdGameName'
17
'riotIdGameName'
18
'riotIdGameName'
19
'riotIdGameName'
20
'riotIdGameName'
21
'riotIdGameName'
22
'riotIdGameName'
23
'riotIdGameName'
24
'riotIdGameName'
25
'riotIdGameName'
26
'riotIdGameName'
27
'riotIdGameName'
28
'riotIdGameName'
29
'riotIdGameName'
30
'riotIdGameName'
31
'riotIdGameName'
32
'riotIdGameName'
33
'riotIdGameName'
34
'riotIdGameName'
35
'riotIdGameName'
36
'riotIdGameName'
37
'riotIdGameName'
38
'riotIdGameName'
39
'riotIdGameName'
40
'riotIdGameName'
41
'riotIdGameName'
42
'riotIdGameName'
43
'riotIdGameName'
44
'riotIdGameName'
45
'riotIdGameName'
46
'riotIdGameName'
47
'riotIdGameName'
48
'riotIdGameName'
49
'riotIdGameName'
50
'riotIdGameName'
51
'riotIdGameName'
52
'riotIdGameName'
53
'riotIdGameName'
54
'riotIdGameNam

In [14]:
df.to_csv("Tims-match-data.csv", index=False)